### Import Modules

In [182]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [183]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

### Input Data

In [184]:
Recyling_Products = [12000, 8000, 19000, 21000, 25000, 5000]
Construction_Costs = [12000, 19500, 32000]
Handling_Charges = [6, 2, 8]
Processing_Capacity = [0, 38000, 18100]

Cik = [
    [1, 9, 9, 4, 9],
    [8, 4, 2, 7, 4],
    [8, 3, 6, 6, 6],
    [1, 2, 5, 4, 4],
    [9, 6, 4, 1, 8],
    [6, 3, 2, 2, 4]
]

Ckl = [
    [8, 9, 7, 9],
    [11, 8, 4, 10],
    [9, 3, 5, 4],
    [5, 4, 8, 6],
    [7, 6, 3, 8]
]

Cli = [
    [6, 5, 4, 6, 7, 2],
    [4, 2, 6, 3, 7, 2],
    [2, 3,1, 5, 8, 2], 
    [7, 4, 5, 7, 5, 9]
]


# Time window in hours
Tij = [
    [14, 10, 15, 10, 5],
    [9, 8, 10, 12, 15],
    [12, 7, 16, 14, 13],
    [3, 13, 15, 22, 14],
    [11, 5, 14, 10, 13],
    [2, 14, 8, 11, 14]
]

Tkl = [
    [11, 11, 13, 8],
    [4, 12, 15, 7],
    [13, 14, 12, 6],
    [5, 12, 10, 14],
    [12, 6, 14, 11],
    [6, 5, 15, 12]
]

Tlj = [
    [10, 7, 12, 13, 10, 12],
    [12, 10, 6, 15, 13, 11],
    [13, 12, 13, 5, 15, 10],
    [13, 14, 15, 11, 12, 10]
]

# Penalty in dollars
Pijk = [
    [5, 4, 6, 3, 7],
    [6, 10, 7, 4, 6],
    [4, 6, 5, 7, 4],
    [7, 5, 6, 4, 5],
    [5, 6, 6, 7, 5],
    [6, 8, 5, 10, 6]
]

Pikl = [
    [4, 5, 6, 7],
    [5, 3, 4, 6],
    [3, 4, 3, 10],
    [2, 4, 5, 5],
    [1, 4, 7, 8],
    [6, 7, 5, 5]
]

Plji = [
    [8, 4, 6, 7, 6, 5],
    [4, 6, 3, 6, 8, 4],
    [6, 5, 7, 7, 5, 6],
    [7, 6, 4, 5, 4, 7]
]


### Creating Model and Declaring Variables

In [185]:

model = gp.Model("closed_loop_supply_chain")


Customer_Centres = model.addVars(5, vtype = gp.GRB.BINARY, name = 'Customer_Centres')
Split_Testing_Centres = model.addVars(6, vtype=gp.GRB.BINARY, name="Split_Testing_Centres")
Remanufacturing_Plants = model.addVars(4, vtype=GRB.BINARY, name="Remanufacturing_Plants")


quantity_ijk = model.addVars(6, 5, name="quantity_ijk")
quantity_ikl = model.addVars(5, 4, name="quantity_ikl")
quantity_lji = model.addVars(4, 6, name="quantity_lji")

 
time_ijk = model.addVars(6, 5, name="time_ijk")
time_ikl = model.addVars(5, 4, name="time_ikl") 
time_lji = model.addVars(4, 6, name="time_lji") 

                            
over_time_ijk = model.addVars(6, 5, name="over_time_ijk")
over_time_ikl = model.addVars(5, 4, name="over_time_ikl")
over_time_lji = model.addVars(4, 6, name="over_time_lji")


### Objective Function

In [186]:
Construction_Cost_Centres = gp.quicksum(Construction_Costs[0] * Customer_Centres[i] for i in range (5))
Construction_Cost_Split = gp.quicksum(Construction_Costs[1] * Split_Testing_Centres[j] for j in range(6))
Construction_Cost_Remanufacturing = gp.quicksum(Construction_Costs[2] * Remanufacturing_Plants[l] for l in range(4))


Handling_Cost_1 = gp.quicksum(Handling_Charges[0] * quantity_ijk[i, j] for i in range(6) for j in range(5))
Handling_Cost_2 = gp.quicksum(Handling_Charges[1] * quantity_ikl[j, k] for j in range(5) for k in range(4))
Handling_Cost_3 = gp.quicksum(Handling_Charges[2] * quantity_lji[k, l] for k in range(4) for l in range(6))


Processing_Cost_1 = gp.quicksum(Cik[i][j] * quantity_ijk[i, j] for i in range(6) for j in range(5))
Processing_Cost_2 = gp.quicksum(Ckl[j][k] * quantity_ikl[j, k] for j in range(5) for k in range(4))
Processing_Cost_3 = gp.quicksum(Cli[k][i] * quantity_lji[k, i] for k in range(4) for i in range(6))


Penalty_Cost_1 = gp.quicksum(Pijk[i][j] * over_time_ijk[i, j] for i in range(6) for j in range(5))
Penalty_Cost_2 = gp.quicksum(Pikl[j][k] * over_time_ikl[j, k] for j in range(5) for k in range(4))
Penalty_Cost_3 = gp.quicksum(Plji[k][i] * over_time_lji[k, i] for k in range(4) for i in range(6))


model.setObjective(Construction_Cost_Split + Construction_Cost_Remanufacturing + 
               Handling_Cost_1 + Processing_Cost_1 + Processing_Cost_2 + Processing_Cost_3 + Construction_Cost_Centres + 
               Handling_Cost_2 + Handling_Cost_3  +
                   Penalty_Cost_1 + Penalty_Cost_2 + Penalty_Cost_3
               , GRB.MINIMIZE)


### Constraints and Optimizing the Model

In [187]:

for j in range(5):
    model.addConstr(gp.quicksum(quantity_ijk[i, j] for i in range(5)) <= Processing_Capacity[1] * Split_Testing_Centres[j])

for k in range(4):
    model.addConstr(gp.quicksum(quantity_ikl[j, k] for j in range(4)) <= Processing_Capacity[2] * Remanufacturing_Plants[k])

for i in range(6):
    model.addConstr(gp.quicksum(quantity_ijk[i, j] for j in range(3)) == Recyling_Products[i])


for k in range(4):
    model.addConstr(gp.quicksum(quantity_ikl[j, k] for j in range(4)) == gp.quicksum(quantity_lji[k, i] for i in range(5)))
    

for i in range(4):
    model.addConstr(Customer_Centres[i] <=1)

for j in range(5):
    model.addConstr(Split_Testing_Centres[j] <= 1)
    
for k in range(4):
    model.addConstr(Remanufacturing_Plants[k] <= 1)

    

        
model.addConstr(gp.quicksum(Split_Testing_Centres[j] for j in range(3)) >= 1)
model.addConstr(gp.quicksum(Remanufacturing_Plants[k] for k in range(4)) >= 1)
model.addConstr(gp.quicksum(Customer_Centres[i] for i in range(4)) >= 1)




# Added New Constraints Below 
model.addConstrs((time_ijk[j, k] >= 0 for j in range(6) for k in range(5)))
model.addConstrs((time_ikl[k, l] >= 0 for k in range(5) for l in range(4)))
model.addConstrs((time_lji[l, j] >= 0 for l in range(4) for j in range(6)))

M = 300000000
model.addConstrs((time_ijk[j, k] <= M * quantity_ijk[j, k] for j in range(6) for k in range(5)))
model.addConstrs((time_ikl[k, l] <= M * quantity_ikl[k, l] for k in range(5) for l in range(4)))
model.addConstrs((time_lji[l, j] <= M * quantity_lji[l, j] for l in range(4) for j in range(6)))


for j in range(5):
    for k in range(4):
        model.addConstr(quantity_ikl[j,k] >=1)
     

model.optimize()



Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 203 rows, 237 columns and 370 nonzeros
Model fingerprint: 0x1b338598
Variable types: 222 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+08]
  Objective range  [1e+00, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1247890.0000
Presolve removed 194 rows and 219 columns
Presolve time: 0.00s
Presolved: 9 rows, 18 columns, 36 nonzeros
Found heuristic solution: objective 1247846.0000
Variable types: 15 continuous, 3 integer (3 binary)

Root relaxation: objective 9.399644e+05, 4 iterations, 0.00 seconds (0.00 work units